In [1]:
# Ensure project root (OpenCEP) is on sys.path for imports like `from base.Pattern import Pattern`
import sys, os, pathlib

nb_dir = (
    pathlib.Path(__file__).parent if "__file__" in globals() else pathlib.Path.cwd()
)
project_root = str((nb_dir / "..").resolve())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# OpenCEP Demo
This notebook demonstrates a simple ascending peak price pattern over Google stock updates.

In [2]:
from datetime import timedelta
from CEP import CEP
from base.Pattern import Pattern
from base.PatternStructure import SeqOperator, PrimitiveEventStructure
from condition.CompositeCondition import AndCondition
from condition.BaseRelationCondition import SmallerThanCondition
from condition.Condition import Variable, SimpleCondition
from stream.FileStream import FileInputStream, FileOutputStream
from plugin.stocks.Stocks import MetastockDataFormatter
import test

This pattern is looking for a short ascend in the Google peak prices:
```
PATTERN SEQ(GoogleStockPriceUpdate a, GoogleStockPriceUpdate b, GoogleStockPriceUpdate c)
WHERE a.PeakPrice < b.PeakPrice AND b.PeakPrice < c.PeakPrice
WITHIN 3 minutes
```

In [3]:
googleAscendPattern = Pattern(
    SeqOperator(
        PrimitiveEventStructure("GOOG", "a"),
        PrimitiveEventStructure("GOOG", "b"),
        PrimitiveEventStructure("GOOG", "c"),
    ),
    AndCondition(
        SmallerThanCondition(
            Variable("a", lambda x: x["Peak Price"]),
            Variable("b", lambda x: x["Peak Price"]),
        ),
        SmallerThanCondition(
            Variable("b", lambda x: x["Peak Price"]),
            Variable("c", lambda x: x["Peak Price"]),
        ),
    ),
    timedelta(minutes=3),
)

Another way to define the above example is to use SimpleCondition and a lambda function:

In [4]:
googleAscendPattern = Pattern(
    SeqOperator(
        PrimitiveEventStructure("GOOG", "a"),
        PrimitiveEventStructure("GOOG", "b"),
        PrimitiveEventStructure("GOOG", "c"),
    ),
    SimpleCondition(
        Variable("a", lambda x: x["Peak Price"]),
        Variable("b", lambda x: x["Peak Price"]),
        Variable("c", lambda x: x["Peak Price"]),
        relation_op=lambda x, y, z: x < y < z,
    ),
    timedelta(minutes=3),
)

Creating a CEP object for monitoring the first pattern from the example above:

In [5]:
cep = CEP([googleAscendPattern])

Creating evaluation manager...
 - Parallel execution: None
 - Storage: None
 - Using ParallelExecutionModes.SEQUENTIAL execution mode
!!! Using default evaluation mechanism parameters...
!!! Creating tree-based evaluation mechanism...
Tree calling create_storage_unit with storage_params: TreeStorageParameters(sort_storage=False, attributes_priorities={}, clean_up_interval=10, prioritize_sorting_by_timestamp=True, enable_load_shedding=False, load_shedding_threshold=1000, load_shedding_drop_rate=0.1, load_shedding_strategy=random)
InternalNode creating storage: sort=False, sorting_key=None
UnsortedPatternMatchStorage created with storage_params: TreeStorageParameters(sort_storage=False, attributes_priorities={}, clean_up_interval=10, prioritize_sorting_by_timestamp=True, enable_load_shedding=False, load_shedding_threshold=1000, load_shedding_drop_rate=0.1, load_shedding_strategy=random)
InternalNode creating storage: sort=False, sorting_key=None
UnsortedPatternMatchStorage created with s

Defining a new file-based event stream formatted according to Metastock 7 format:

In [6]:
events = FileInputStream("../test/EventFiles/NASDAQ_SHORT.txt")

Applying an existing CEP object on an event stream created above and storing the resulting pattern matches to a file:

In [7]:
cep.run(
    events,
    FileOutputStream("../test/demo/Matches", "output.txt"),
    MetastockDataFormatter(),
)

Starting CEP evaluation...
Using generic file input stream processing
Playing new event on tree: {'Stock Ticker': 'GOOG', 'Date': 200802010900, 'Opening Price': 532.04, 'Peak Price': 532.04, 'Lowest Price': 530.51, 'Close Price': 530.51, 'Volume': 17665}, trying to find matches
Playing new event. Event types listeners: {'GOOG': [<tree.nodes.LeafNode.LeafNode object at 0x000001EB3420CD10>, <tree.nodes.LeafNode.LeafNode object at 0x000001EB3420D490>, <tree.nodes.LeafNode.LeafNode object at 0x000001EB3420D970>]}
SortedPatternMatchStorage.add() called Key: 2008-02-01 09:00:00, length of partial matches: 0
Current events in pattern match: [{'Stock Ticker': 'GOOG', 'Date': 200802010900, 'Opening Price': 532.04, 'Peak Price': 532.04, 'Lowest Price': 530.51, 'Close Price': 530.51, 'Volume': 17665}]
SortedPatternMatchStorage.add() called Key: 2008-02-01 09:00:00, length of partial matches: 0
Current events in pattern match: [{'Stock Ticker': 'GOOG', 'Date': 200802010900, 'Opening Price': 532.04

0.090934